In [ ]:
import json
import warnings

# suppress display of warnings
warnings.filterwarnings('ignore')
with open("code/dataset/train_data.json", "r") as f:
    train_data = json.load(f)

with open("code/dataset/test_data.json", "r") as f:
    test_data = json.load(f)

print(len(train_data))


In [ ]:
import os
import numpy as np
import pandas as pd
import sagemaker

# Provides a collection of methods for working with SageMaker resources
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-steam-sa"
region = sagemaker_session.boto_session.region_name
role = sagemaker.get_execution_role()


In [ ]:
inputs_train = sagemaker_session.upload_data(
    "code/dataset/train_data.json", bucket=bucket, key_prefix=prefix)
inputs_test = sagemaker_session.upload_data(
    "code/dataset/test_data.json", bucket=bucket, key_prefix=prefix)


In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.huggingface import HuggingFace

# 1. Defining the estimator

estimator = PyTorch(entry_point="train.py",
                    source_dir="code",
                    role=role,
                    framework_version="1.9",
                    py_version="py38",
                    instance_count=1,
                    # Type of instance we want the training to happen
                    instance_type="ml.m5.xlarge",
                    hyperparameters={"epochs": 2,
                                     "num_labels": 2,
                                     }
                    )

estimator.fit({"training": inputs_train, "testing": inputs_test})
